In [1]:
import sys
import os
sys.path.insert(0, '../icenumerics')
import icenumerics as ice
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tqdm.notebook as tqdm

ureg = ice.ureg

from importlib import reload  
import shakti as sha
reload(sha)

from icenumerics.geometry import ordering 

%load_ext autoreload
%autoreload 2


import csv as csv
#import tqdm.auto as tqdm
from multiprocessing import Pool

In [2]:
output = "/home/carolina/CairoLattice_ACI_PROJECT/Scripts/Output_Script-12.0"

# Shakti Lattice 

    In this script, I will run simulations with open boundaries. I will run simulations in parallel.

In [3]:
def do_everything(exp_entry):
    
    e = exp_entry[1].e 
    l = exp_entry[1].l
    
    trap_sep = 10 # Short trap
    radius = 5
    distance = 3
    interparticle_distance = 2*radius + distance # distance among a particle next to the other 
    part_d = interparticle_distance*np.cos(np.pi/4) # distance among a particle in front of the other 
    plaquette_cte = part_d*4+2*trap_sep

    sp = sha.spins()
    ureg = ice.ureg

    sp.create_lattice(geometry = "shakti",size =[10,10],lattice_constant=plaquette_cte*ureg.um , border="periodic")
    sp.order_spins(ordering.random_ordering)

    particle = ice.particle(radius = 5*ureg.um,
                 susceptibility = 0.005,
                 diffusion = 0.125*ureg.um**2/ureg.s,
                 temperature = 300*ureg.K,
                 density = 1000*ureg.kg/ureg.m**3)

    trap_short = ice.trap(trap_sep = 10*ureg.um,
                   height = 25*ureg.pN*ureg.nm,
                   stiffness = 0.1*ureg.fN/ureg.nm)

    trap_long = ice.trap(trap_sep =(20+13*np.sqrt(2))*ureg.um,
                   height = 25*ureg.pN*ureg.nm,
                   stiffness = 0.1*ureg.fN/ureg.nm)

    traps = []

    for s in sp:

        if ( s.direction[0].magnitude >= plaquette_cte or s.direction[0].magnitude <= -plaquette_cte or 
            s.direction[1].magnitude >= plaquette_cte or s.direction[1].magnitude <= -plaquette_cte):

            traps.append(trap_long)

        else:

            traps.append(trap_short)

    col = ice.colloidal_ice(sp, particle, traps, height_spread = 0, susceptibility_spread = 0.1)
    col.pad_region(5*ureg.um)

    world = ice.world(
        field = 25*ureg.mT,
        temperature = 300*ureg.K,
        dipole_cutoff = 200*ureg.um)

    change_m_time = 2000*ureg.s
    total_time = 2500*ureg.s
    col.simulate(world,
                 name = "ACI_ShaktiLattice_25mT-2500s_FreeBound_l%u_exp%u"%(l,e),
                 include_timestamp = False,
                 targetdir = output,
                 framerate = 1*ureg.Hz,
                 timestep = 10*ureg.ms,
                 run_time = total_time,
                 output = ["x","y","z","mux","muy","muz"])

    col.sim.field.fieldz = "(time<2000000000)*v_Bmag*time/%f+(time>=2000000000)*v_Bmag"%(change_m_time.to(ureg.us).magnitude)
    col.run_simulation() 
    col.load_simulation(slice(0,None,1))
    
    # <Generating the vertex array>
    edges_shakti_values = pd.read_csv('edges_shakti_10x10.dat')
    vertices = pd.read_csv('vertices_shakti_10x10.dat')

    v_non_inferred = ice.vertices(
        positions = vertices[["x","y"]].values,
        edges = edges_shakti_values[["start","stop"]].values)
    
    # <Extract vertices dataframe and save it>
    frames = col.trj.index.get_level_values("frame").unique()

    v_df = []

    for f in tqdm.tqdm(frames[::5]):
        col.set_state_from_frame(f)
        
        v_non_inferred.update_directions(col)
        v_non_inferred.classify_vertices(ice.ice_to_spins(col))
        vertices = v_non_inferred.vertices

        v_df.append(vertices.copy())


    v_df = pd.concat(v_df,keys=frames[::5], names = ["frame"])

    v_df.to_csv(os.path.join(output,"ACI_ShaktiLattice_25mT-2500s_FreeBound_l%u_exp%u.dat"%(l,e)), sep='\t')

In [4]:
e =  np.arange(10,50)
l =  [10]
L, E = np.meshgrid(l,e)
experiments = pd.DataFrame({"e":E.flatten(),"l":L.flatten()})
experiments

,e,l
0,10,10
1,11,10
2,12,10
3,13,10
4,14,10
5,15,10
6,16,10
7,17,10
8,18,10
9,19,10


In [ ]:
# %%time
if __name__ ==  '__main__': 
    num_processors = 5
    p=Pool(processes = num_processors)
    
    ## Create index text file
    if not os.path.exists(output):
        os.makedirs(output)
    with open(os.path.join(output,"index_ACI_ShaktiLattice_25mT-2500s_FreeBound.dat"),'w',newline='') as file:
        writer = csv.writer(file,delimiter='\t')
        writer.writerow(["filename", "l", "exp"])
        
    list(tqdm.tqdm(p.imap(do_everything, experiments.iterrows()), total=len(experiments)))

/home/carolina/CairoLattice_ACI_PROJECT/Scripts/shakti.py:251: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_centers[:,1] = centersY
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/shakti.py:251: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_centers[:,1] = centersY
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/shakti.py:251: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_centers[:,1] = centersY
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/shakti.py:251: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_centers[:,1] = centersY
/home/carolina/CairoLattice_ACI_PROJECT/Scripts/shakti.py:251: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_centers[:,1] = centersY


scanning file


scanning file


scanning file


scanning file


...

scanning file


scanning file


scanning file


scanning file


scanning file
